<a href="https://colab.research.google.com/github/semmatoninn/PLGA-Microparticles-Drug-Release/blob/main/PLGAmicroparticle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Data Preparation

In [ ]:
RANDOM_SEED = 1147

In [ ]:
import pandas as pd
import numpy as np

!pip install rdkit
from rdkit import Chem # module chem
from rdkit.Chem import Draw, Descriptors
from rdkit.Chem import PandasTools # module from rdkit.Chem

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    max_error
)

from sklearn.linear_model import LinearRegression

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV, KFold, GroupKFold, RandomizedSearchCV
from sklearn.kernel_ridge import KernelRidge

from xgboost import XGBRegressor

!pip install shap
import shap

In [ ]:
!git clone https://github.com/semmatoninn/PLGA-Microparticles-Drug-Release.git
%cd PLGA-Microparticles-Drug-Release
import pandas as pd
df = pd.read_excel("mp_dataset_initial.xlsx")
df.head()


In [ ]:
df.shape

In [ ]:
df.columns


In [ ]:
df.info() # 17 columns

In [ ]:
df.describe()

In [ ]:
df.head(10)

**Dropping Source link Column - No predictive value**

In [ ]:
df = df.drop(columns = 'DOI')
print(f"After dropping columns: the number of columns are {df.shape[1]}")

## **Null Values**

In [ ]:
print(f"The number of nulls in df: {df.isna().sum().sum()}")

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
# Drop columns with too many missing values - 4 columns dropped
df = df.drop(columns=['PDI', "Polymer Mn","Polymer Mw ","Polymer Molecular Weight"])
print(f"After dropping columns: the number of columns are {df.shape[1]}")

## Duplicates

In [ ]:
print(f"Number of duplicated rows: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True) # dropping duplicates

## **Zero variance columns**

In [ ]:
zero_var_cols = df.columns[df.nunique() == 1].tolist()
print("Zero-variance columns:", zero_var_cols)


In [ ]:
df.info()

In [ ]:
df.head(11)

## **Canonicalization**

In [ ]:
def canonicalize_smiles(smiles, isomeric:bool=True): # isomeric is flag/true or false, true is default
    """
    Convert any SMILES to its canonical form.

    Args:
        smiles (str): Input SMILES string

    Returns:
        str: Canonical SMILES
    """
    mol = Chem.MolFromSmiles(smiles) # creating mol object
    if mol is None:
        return None  # Invalid SMILES
    return Chem.MolToSmiles(mol, isomericSmiles=isomeric)

In [ ]:
df['Drug canonical_smiles'] = df['Drug SMILES'].apply(canonicalize_smiles)

In [ ]:
df.head()

In [ ]:
# Add a column called 'molecule' with mol objects
PandasTools.AddMoleculeColumnToFrame(df, # AddMoleculeColumnToFrame is a function inside PandasTools module
                                     smilesCol='Drug canonical_smiles', # input column
                                     molCol='Drug molecule' # output column - new column created
                                     ) # creates mol object from canonical smiles column

# df['molecule'] = df['canonical_smiles'].apply(Chem.MolFromSmiles) # the previous line is same as this
df.head()

In [ ]:
# Define a function to calculate key molecular descriptors
def calculate_key_descriptors(mol):
    """Calculate key molecular descriptors for a molecule."""

    descriptors = {
        'Drug MW': Descriptors.MolWt(mol),
        'Drug LogP': Descriptors.MolLogP(mol), # Lipophilicity (octanol-water partition coefficient)
        'Drug TPSA': Descriptors.TPSA(mol), # Topological Polar Surface Area (important for drug absorption)
        # 'Drug HBD': Descriptors.NumHDonors(mol),
        # 'Drug HBA': Descriptors.NumHAcceptors(mol)
    }

    return descriptors

# Apply the function to the molecules in the DataFrame
descriptor_data = []
for idx, mol in df['Drug molecule'].items(): # index is row index, mol is molecule column value, molecule is mol object, it is pd series, index value pair
    desc = calculate_key_descriptors(mol) # dictionary created for each sample molecule
    descriptor_data.append(desc)

# Create descriptor DataFrame with pd.DataFrame()
# We use the same index as the original DataFrame to maintain row alignment
descriptor_df = pd.DataFrame(descriptor_data, index=df.index) # new dataframe created with descriptor columns

# Combine with the original molecules DataFrame
df_with_descriptors = pd.concat([df, descriptor_df], axis=1) # axis = 1, side by side concatenation

# Check the new dimensions - we should have the same number of rows but more columns
print(f"Dataset shape: {df_with_descriptors.shape}")
df_with_descriptors.head()

## Train test split

In [ ]:
print(df.columns.tolist())


In [ ]:
# One-hot encode Formulation Method
TARGET = "Release "
df_encoded = pd.get_dummies(
    df_with_descriptors,
    columns=["Formulation Method"],
    drop_first=True
)

# Define FEATURES (exclude target + string/RDKit fields)
exclude_cols = [
    TARGET,
    "Drug",
    "Drug SMILES",
    "Drug canonical_smiles",
    "Drug molecule",
    "Formulation Index"
]

FEATURES = [col for col in df_encoded.columns if col not in exclude_cols]

# Group-aware train/test split (no leakage)
from sklearn.model_selection import GroupShuffleSplit

groups = df_encoded["Formulation Index"]

gss = GroupShuffleSplit(test_size=0.2, random_state=RANDOM_SEED)

train_idx, test_idx = next(gss.split(df_encoded, groups=groups))

#  Final datasets
X_train = df_encoded.iloc[train_idx][FEATURES]
X_test  = df_encoded.iloc[test_idx][FEATURES]

y_train = df_encoded.iloc[train_idx][TARGET]
y_test  = df_encoded.iloc[test_idx][TARGET]

# Looking at shapes
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


## Correlation analysis

### Pearson

In [ ]:
df_corr = X_train.copy()
df_corr["Release "] = y_train.values  # Add target column

# Compute correlation
corr_train = df_corr.corr(method="pearson")

plt.figure(figsize=(14, 12))

# Mask upper triangle for cleaner plot
mask = np.triu(np.ones_like(corr_train, dtype=bool))

sns.heatmap(
    corr_train,
    mask=mask,
    cmap='coolwarm',
    annot=True,   # Set True to show numbers
    center=0,
    square=True,
    linewidths=0.4,
    cbar_kws={'label': 'Correlation Coefficient'}
)

plt.title("Correlation Heatmap (X_train + Release)", fontweight='bold', pad=20)
plt.tight_layout()
plt.show()


In Pearson correlation - linear correlations are captured

Highest correlations between features are in

Drug loading capacity - Initial drug to polymer ratio 0.88

Drug TPSA and MW 0.87

Drug Encapsulation Efficiency and Drug Loading Capacity 0.46

Highest correlation with target variable release is time 0.42



### Spearman

In [ ]:

# Compute Spearman correlation
corr_train = df_corr.corr(method="spearman")

plt.figure(figsize=(14, 12))

# Mask upper triangle for cleaner plot
mask = np.triu(np.ones_like(corr_train, dtype=bool))

sns.heatmap(
    corr_train,
    mask=mask,
    cmap='coolwarm',
    annot=True,   # Set True to show numbers
    center=0,
    square=True,
    linewidths=0.4,
    cbar_kws={'label': 'Correlation Coefficient'}
)

plt.title("Correlation Heatmap (X_train + Release)", fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In Spearman correlations non-linear correlations are also captured, monotonic relationships are captured

Highest correlated features are

Drug loading capacity and initial drug to polymer ratio id 0.91

Drug TPSA and MW correlation is 0.62

Drug encapsulation efficiency and drug loading capacity correlation is 0.49

Drug LogP and Solubility Enhancer Concentration correlation is 0.43

Release and time correlation is 0.67


## Feature Outliers

In [ ]:
numeric_cols = X_train.select_dtypes(include=[np.number]).columns

plt.figure(figsize=(12, 6))
sns.boxplot(X_train[numeric_cols], orient='h')
# plt.tight_layout()
plt.savefig("outlier_detection.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
## regular split
#from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(
#    df[FEATURES], df[TARGET], test_size=0.2, random_state=RANDOM_SEED
#)

# II. Modelling

## Regression

In [ ]:

def get_regression_metrics(model, X, y_true):
    """
    Computes stable regression metrics for drug-release prediction.
    MAPE is intentionally excluded because y contains zeros - it explodes.
    """
    # Predictions
    y_pred = model.predict(X)

    # Core metrics
    r2  = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mxe = max_error(y_true, y_pred)

    return {
        "r2": r2,
        "mse": mse,
        "mae": mae,
        "max_error": mxe
    }


### Linear Regression - Baseline

In [ ]:

linear_regressor = LinearRegression()

# Train the model on the training data
linear_regressor.fit(X_train,y_train)

linear_regressor_results_train = get_regression_metrics(linear_regressor,X_train,y_train )
linear_regressor_results_test = get_regression_metrics(linear_regressor, X_test,y_test )

# Print the results
print("Linear Regression Results - Train Set:", linear_regressor_results_train)
print("Linear Regression Results - Test Set:", linear_regressor_results_test)

### Linear Regression with Correlation based feature selection and scaler optimization








In [ ]:

# Correlation-based feature filter

class CorrelationFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.9):
        self.threshold = threshold
        self.to_drop_ = None

    def fit(self, X, y=None):
        X_df = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)

        # Pearson absolute correlation
        corr = X_df.corr().abs()

        # Upper triangle (no diag)
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

        # Choose features to drop
        self.to_drop_ = [
            column
            for column in upper.columns
            if any(upper[column] > self.threshold)
        ]
        return self

    def transform(self, X):
        X_df = X if isinstance(X, pd.DataFrame) else pd.DataFrame(X)
        return X_df.drop(columns=self.to_drop_, errors="ignore")


#
# Pipeline: Corr Filter , Scaler , Linear Regression
#    (scaler will be chosen via GridSearch)

pipe = Pipeline([
    ("corr_filter", CorrelationFilter()),
    ("scaler", StandardScaler()),   # placeholder, will be overridden by param_grid
    ("model", LinearRegression())
])

#  Hyperparameter grid
#    correlation threshold
#    scaler type: Standard, MinMax, or no scaling

param_grid = {
    "corr_filter__threshold": [0.75, 0.8, 0.85, 0.9, 0.95],
    "scaler": [
        StandardScaler(),
        MinMaxScaler(),
        "passthrough"   # no scaling
    ]
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="r2",
    cv=5,
    n_jobs=-1,
    verbose=1
)


#  Fit on training data
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV R²:", grid.best_score_)

best_model = grid.best_estimator_

linear_corr_train = get_regression_metrics(best_model, X_train, y_train)
linear_corr_test  = get_regression_metrics(best_model, X_test, y_test)

print("Corr-Filtered + Scaler + Linear Regression - Train:", linear_corr_train)
print("Corr-Filtered + Scaler + Linear Regression - Test :", linear_corr_test)

best_filter = best_model.named_steps["corr_filter"]
print("Dropped features at best threshold:", best_filter.to_drop_)

print("Best scaler:", best_model.named_steps["scaler"])


Performance increased very slightly

### Linear Regression with Forward feature selection and scaler optimization



In [ ]:
# Base estimator
base_model = LinearRegression()

# Function to generate a forward selector with a given number of features
def make_forward_selector(k):
    return SequentialFeatureSelector(
        estimator=base_model,
        n_features_to_select=k,
        direction="forward",
        scoring="r2",
        cv=KFold(n_splits=5, shuffle=True, random_state=42),
        n_jobs=-1
    )

pipe = Pipeline([
    ("scaler", StandardScaler()),      # placeholder (GridSearch will override)
    ("feature_select", make_forward_selector(5)),  # placeholder
    ("model", LinearRegression()),
])


param_grid = {
    "scaler": [
        StandardScaler(),
        MinMaxScaler(),
        "passthrough"              # no scaling
    ],
    "feature_select": [
        make_forward_selector(5),
        make_forward_selector(8),
        make_forward_selector(10),
        make_forward_selector(12)
    ],
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="r2",
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Fit the full optimized pipeline
grid.fit(X_train, y_train)

# Best model and parameters
print("Best Params:", grid.best_params_)
print("Best CV R²:", grid.best_score_)

best_model = grid.best_estimator_


train_metrics = get_regression_metrics(best_model, X_train, y_train)
test_metrics  = get_regression_metrics(best_model, X_test, y_test)

print("\nOptimized Forward Selection + Scaler Pipeline")
print("Train:", train_metrics)
print("Test :", test_metrics)


sfs = best_model.named_steps["feature_select"]
selected_features = X_train.columns[sfs.get_support()]

print("\nSelected features:")
for f in selected_features:
    print("  -", f)


Did not perform better

##KRR

In [ ]:

krr = KernelRidge(kernel='rbf') # without regularization and no scaling
krr.fit(X_train,y_train)
# get the metrics on the train and the test set using the get_regression_metrics functions (as above)
krr_results_train = get_regression_metrics(krr, X_train,y_train )
krr_results_test = get_regression_metrics(krr, X_test,y_test )


# the results
print("KRR Results - Train Set:", krr_results_train)
print("KRR Results - Test Set:", krr_results_test)

without regularization, KRR usually overfits, shown here as well

In [ ]:
pipe_w_scaling = Pipeline(
   [
       ('scaling', StandardScaler()),
       ('krr', KernelRidge(kernel="rbf"))
   ]
)
pipe_w_scaling.fit(X_train,y_train)

pipeline_results_train = get_regression_metrics(pipe_w_scaling, X_train, y_train )
pipeline_results_test = get_regression_metrics(pipe_w_scaling, X_test,y_test )


# # Print the results
print("Pipeline Results - Train Set:", pipeline_results_train)
print("Pipeline Results - Test Set:", pipeline_results_test)

In [ ]:
print(pipe_w_scaling.named_steps["krr"].get_params())

Mild overfitting for standardized default krr model

### KRR with Regularization and Hyperparameter Tuning

In [ ]:


# possible scalers
scalers = [StandardScaler(), MinMaxScaler(), "passthrough"]

pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('krr', KernelRidge(kernel='rbf'))
])

param_grid = [
    {
        'scaling': scalers,
        'krr__alpha': [0.001, 0.01, 0.1, 1],
        'krr__gamma': np.logspace(-3, 1, 20)
    }
]

# Grid search with 5-fold CV
search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

search.fit(X_train, y_train)

# Print best result
print("Best Params:", search.best_params_)
print("Best CV R²:", search.best_score_)


Fitting 5 folds for each of 240 candidates, totalling 1200 fits


In [ ]:
X_train.columns

In [ ]:
best_krr = search.best_estimator_
best_krr.fit(X_train, y_train)

train_results = get_regression_metrics(best_krr, X_train, y_train)
test_results  = get_regression_metrics(best_krr, X_test,  y_test)

print("Tuned KRR - Train:", train_results)
print("Tuned KRR - Test :", test_results)


better than linear regression models

In [ ]:
# group aware cross validation

# groups for TRAIN SET (must align with X_train)
groups_train = df_encoded.iloc[train_idx]["Formulation Index"].values

# Define pipeline placeholder (scaler will be set dynamically)
pipe = Pipeline([
    ('scaling', StandardScaler()),  # dummy; will be overridden in param_grid
    ('krr', KernelRidge(kernel='rbf'))
])

# Define parameter grid
param_grid = {
    'scaling': scalers,
    'krr__alpha': [0.001, 0.01, 0.1, 1],
    'krr__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10]
}

# Group-aware CV instead of plain cv=5
cv = GroupKFold(n_splits=5)

# Grid search with group-aware CV
search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=cv,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

search.fit(X_train, y_train, groups=groups_train)

print("Best Params:", search.best_params_)
print("Best CV R²:", search.best_score_)

best_krr = search.best_estimator_
best_krr.fit(X_train, y_train)

train_results = get_regression_metrics(best_krr, X_train, y_train)
test_results  = get_regression_metrics(best_krr, X_test,  y_test)

print("Tuned KRR - Train:", train_results)
print("Tuned KRR - Test :", test_results)

## XGBoost

In [ ]:
# XGBoost expects floats
X_tr = X_train.to_numpy(dtype=float)
y_tr = y_train.to_numpy(dtype=float)
X_te = X_test.to_numpy(dtype=float)
y_te = y_test.to_numpy(dtype=float)



xgb = XGBRegressor(
    n_estimators=1200,        # number of trees
    learning_rate=0.04,       # how much each tree contributes
    # max_depth=5,              # trees depth
    # min_child_weight=12,      # larger leaves (regularization)
    gamma=0.6,                # split penalty
    # subsample=0.7,            # row sampling
    # colsample_bytree=0.6,     # feature sampling
    reg_alpha=1.0,            # L1
    reg_lambda=6.0,           # L2
    objective="reg:squarederror",
    tree_method="hist",
    n_jobs=4,
    random_state=1147,
    verbosity=0,
    max_depth=4,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8
)

xgb.fit(X_tr, y_tr)

print("XGBoost Results - Train Set:", get_regression_metrics(xgb, X_tr, y_tr))
print("XGBoost Results - Test Set:",  get_regression_metrics(xgb, X_te, y_te))

Xgboost performance shows that this is a difficult task to predict

### Hypertune XGBOOST

In [ ]:
# from xgboost import XGBRegressor
# from sklearn.model_selection import GroupKFold, RandomizedSearchCV  # or GridSearchCV
# from sklearn.metrics import make_scorer, r2_score


groups_train = df_encoded.iloc[train_idx]["Formulation Index"].values

print("Unique formulation groups in train:", np.unique(groups_train).shape[0])

xgb = XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",
    random_state=RANDOM_SEED,
    n_jobs=-1
)

param_distributions = {
    "n_estimators": [300, 500, 800, 1200],
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "min_child_weight": [1, 3, 5, 10],
    "gamma": [0, 0.1, 0.3, 0.5],
    "reg_alpha": [0, 0.01, 0.1, 1],
    "reg_lambda": [0.1, 1, 5, 10],
}

# 4) Group-aware CV
cv = GroupKFold(n_splits=5)

# 5) RandomizedSearchCV (you can swap to GridSearchCV if you prefer)
search_xgb = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_distributions,
    n_iter=50,              # adjust depending on time
    scoring="r2",
    cv=cv,
    n_jobs=-1,
    verbose=2,
    random_state=RANDOM_SEED
)

search_xgb.fit(X_train, y_train, groups=groups_train)

print("Best XGB Params:", search_xgb.best_params_)
print("Best XGB CV R²:", search_xgb.best_score_)

best_xgb = search_xgb.best_estimator_


xgb_train_results = get_regression_metrics(best_xgb, X_train, y_train)
xgb_test_results  = get_regression_metrics(best_xgb, X_test,  y_test)

print("Tuned Group-Aware XGB - Train:", xgb_train_results)
print("Tuned Group-Aware XGB - Test :", xgb_test_results)


Best model so far

# III. Model Interpretation and Explainability

## PFI

In [ ]:
def get_perm_importance(model, X, y, features, n=10):
    """
    Calculate permutation importance score for each feature
    """
    # Calculate baseline score (without permuting any feature)

    baseline_score = model.score(X, y)

    importance_scores = {}

    # Loop over each feature
    for feature in features:
        X_perm = X.copy()
        sum_score = 0

        # Repeat n times to get average importance score
        for i in range(n):
            # Calculate score when given feature is permuted
            X_perm[feature] = np.random.permutation(X_perm[feature].values)
            permuted_score = model.score(X_perm, y)

            sum_score += permuted_score

        # Calculate decrease in score
        importance_score = baseline_score - (sum_score / n)
        importance_scores[feature] = importance_score

    return importance_scores

In [ ]:

features_no_time = [f for f in X_test.columns if "time" not in f.lower()]

# Compute PFI for best_xgb

print("Calculating permutation importance for best_xgb...")

xgb_perm_importance = get_perm_importance(
    best_xgb,
    X_test,
    y_test,
    features_no_time        # time excluded
)

#  Sort and get top 10 features
sorted_xgb_importance = sorted(
    xgb_perm_importance.items(),
    key=lambda item: item[1],
    reverse=True
)

top_10_xgb_features = [item[0] for item in sorted_xgb_importance[:10]]
top_10_xgb_values   = [item[1] for item in sorted_xgb_importance[:10]]

print("\n--- Top 10 Permutation Feature Importance (best_xgb, Time Excluded) ---")
for f, v in zip(top_10_xgb_features, top_10_xgb_values):
    print(f"{f}: {v:.5f}")


#  Plotting top 10 PFI
plt.figure(figsize=(10, 6))
sns.barplot(
    x=top_10_xgb_values,
    y=top_10_xgb_features,
    palette='viridis'
)
plt.xlabel('Mean Decrease in Score (R²)')
plt.ylabel('Feature')
plt.title('Top 10 Permutation Feature Importance for XGBoost (Time Excluded)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


## XGBOOST with PFI Feature Selection

In [ ]:

# Sort PFI results (highest to lowest)
sorted_importance = sorted(
    xgb_perm_importance.items(),
    key=lambda x: x[1],
    reverse=True
)

# Select Top-10 from PFI
top_10_pfi_features = [feat for feat, score in sorted_importance[:10]]

# Automatically detect REAL time feature
time_cols = [c for c in X_train.columns if "time" in c.lower()]
if len(time_cols) == 0:
    raise ValueError("No time column found in X_train!")
time_col = time_cols[0]

# Add it if missing
if time_col not in top_10_pfi_features:
    top_10_pfi_features.append(time_col)

print("\nSelected features for final model (Top-10 PFI + Time):")
for f in top_10_pfi_features:
    print("  -", f)

# Subset train/test to these features
X_train_top10 = X_train[top_10_pfi_features].copy()
X_test_top10  = X_test[top_10_pfi_features].copy()

#Final XGBoost (using best hyperparameters)
final_xgb_top10 = XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",
    random_state=RANDOM_SEED,
    n_jobs=-1,
    subsample=0.8,
    reg_lambda=0.1,
    reg_alpha=0,
    n_estimators=300,
    min_child_weight=1,
    max_depth=6,
    learning_rate=0.03,
    gamma=0.3,
    colsample_bytree=0.6
)

final_xgb_top10.fit(X_train_top10, y_train)

# Evaluate final model
train_top10_metrics = get_regression_metrics(final_xgb_top10, X_train_top10, y_train)
test_top10_metrics  = get_regression_metrics(final_xgb_top10, X_test_top10,  y_test)

print("\nFinal XGB (Top-10 PFI + Time) - Train:", train_top10_metrics)
print("Final XGB (Top-10 PFI + Time) - Test :", test_top10_metrics)


## SHAP Analysis

In [ ]:
import numpy as np
import shap

#Original SHAP values
explainer = shap.TreeExplainer(final_xgb_top10)
shap_values = explainer(X_train_top10)
shap.plots.bar(shap_values)


## Beeswarm Plot

In [ ]:
import numpy as np
import shap

#Original SHAP values
explainer = shap.TreeExplainer(final_xgb_top10)
shap_values = explainer(X_train_top10)

#Find the time column and its index
time_cols = [c for c in X_train_top10.columns if "time" in c.lower()]
time_col = time_cols[0]          # assume only one time column
time_idx = list(X_train_top10.columns).index(time_col)

#Build a mask that EXCLUDES the time column
mask = np.array([i != time_idx for i in range(len(X_train_top10.columns))])

#Create a new SHAP Explanation without time
shap_values_no_time = shap.Explanation(
    values       = shap_values.values[:, mask],
    base_values  = shap_values.base_values,
    data         = shap_values.data[:, mask],
    feature_names=[name for i, name in enumerate(shap_values.feature_names) if i != time_idx]
)

#Beeswarm plot WITHOUT time
shap.plots.beeswarm(shap_values_no_time)


#IV. Predicted vs Actual Drug release


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# TEST portion of the original dataset ---
test_df = df_with_descriptors.iloc[test_idx].copy()

# Identify formulations present in the test set
forms_in_test = test_df["Formulation Index"].unique()

# Choose 4 representative formulations
form_sizes = test_df["Formulation Index"].value_counts()
chosen_forms = form_sizes.index[:4].tolist()

print("Selected unseen formulations:", chosen_forms)

#plot setup
plt.figure(figsize=(14, 10))

time_col = "Time "       # adjust if you stripped spaces
release_col = "Release " # adjust if you stripped spaces

for i, form in enumerate(chosen_forms, 1):

    # subset data for this formulation
    sub_df = test_df[test_df["Formulation Index"] == form].copy()

    # ensure time order
    sub_df = sub_df.sort_values(by=time_col)

    # slice matching rows from X_test
    X_sub = X_test.loc[sub_df.index, top_10_pfi_features]

    # predictions
    sub_df["Release_pred"] = final_xgb_top10.predict(X_sub)

    # --- 3. Plot each formulation ---
    plt.subplot(2, 2, i)

    plt.plot(sub_df[time_col], sub_df[release_col], "o-", label="Actual")
    plt.plot(sub_df[time_col], sub_df["Release_pred"], "s--", label="Predicted")

    plt.title(f"Formulation {form}")
    plt.xlabel("Time (days)")
    plt.ylabel("Cumulative Release (w/w)")
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
df[df["Formulation Index"] == 123][["Drug"]]

#V. Drug descriptor and Clustering


In [ ]:
df_with_descriptors
import pandas as pd

drug_descriptor_cols = [
    "Drug MW",
    "Drug TPSA",
    "Drug LogP",
    # "Drug pKa"   # uncomment if exists
]

# One row per drug
drug_df = (
    df_with_descriptors
    .groupby("Drug")[drug_descriptor_cols]
    .mean()        # descriptors should be constant, mean is safe
    .reset_index()
)

print(drug_df.head())
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_drug = scaler.fit_transform(drug_df[drug_descriptor_cols])

In [ ]:
import umap

umap_model = umap.UMAP(
    n_neighbors=5,
    min_dist=0.3,
    n_components=2,
    random_state=1147
)

drug_embedding = umap_model.fit_transform(X_drug)

drug_df["UMAP1"] = drug_embedding[:,0]
drug_df["UMAP2"] = drug_embedding[:,1]

In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=3,
    metric="euclidean"
)

drug_df["Cluster"] = clusterer.fit_predict(drug_embedding)

print(drug_df[["Drug","Cluster"]])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,8))

scatter = plt.scatter(
    drug_df["UMAP1"],
    drug_df["UMAP2"],
    c=drug_df["Cluster"],
    cmap="viridis",
    s=80
)

for i, txt in enumerate(drug_df["Drug"]):
    plt.annotate(txt,
                 (drug_df["UMAP1"][i], drug_df["UMAP2"][i]),
                 fontsize=9)

plt.title("Drug Descriptor Space (UMAP)")
plt.xlabel("UMAP1")
plt.ylabel("UMAP2")
plt.colorbar(scatter, label="Cluster")
plt.show()

In [ ]:
cluster_names = {
    0: "Small hydrophobic molecules",
    1: "Large hydrophilic molecules",
    2: "Midsized Amphiphilic",
    3: "Hydrophilic drugs"
}

drug_df["Cluster_Label"] = drug_df["Cluster"].map(cluster_names)

drug_df[["Drug", "Cluster", "Cluster_Label"]].head()

In [ ]:
# Build mapping: Drug → Cluster_Label
drug_to_cluster = drug_df.set_index("Drug")["Cluster_Label"].to_dict()

# Add cluster column to your main dataframe
df_with_descriptors["Cluster_Label"] = df_with_descriptors["Drug"].map(drug_to_cluster)

In [ ]:
train_df = df_with_descriptors.iloc[train_idx].copy()

# Keep only rows used in final model
train_df = train_df.loc[X_train_top10.index]

print(train_df[["Drug","Cluster_Label"]].head())

Understanding which discriptor is important per cluster

In [ ]:
import numpy as np
import pandas as pd
import shap

# 1) Build train_df from original data, then align to X_train_top10 rows
train_df = df_with_descriptors.iloc[train_idx].copy()
train_df = train_df.loc[X_train_top10.index].copy()

# Sanity check
print(train_df[["Drug", "Cluster_Label"]].head())
print("Clusters in train:", train_df["Cluster_Label"].unique())

In [ ]:
explainer = shap.TreeExplainer(final_xgb_top10)
shap_exp = explainer(X_train_top10)   # shap.Explanation# Identify time column (if present) and remove it from the SHAP matrix
feature_names = list(shap_exp.feature_names)

time_cols = [c for c in feature_names if "time" in c.lower()]
if len(time_cols) > 0:
    time_col = time_cols[0]
    time_idx = feature_names.index(time_col)

    keep_mask = np.array([i != time_idx for i in range(len(feature_names))])

    shap_values = shap_exp.values[:, keep_mask]
    X_vals = X_train_top10.values[:, keep_mask]
    kept_features = [f for i, f in enumerate(feature_names) if i != time_idx]
else:
    time_col = None
    shap_values = shap_exp.values
    X_vals = X_train_top10.values
    kept_features = feature_names

print("Removed time column:", time_col)
print("Features used for cluster-SHAP:", kept_features)

In [ ]:
clusters = pd.Series(train_df["Cluster_Label"].values, index=X_train_top10.index)

cluster_shap = {}

for cl in sorted(clusters.dropna().unique()):
    idx = (clusters == cl).values
    mean_abs = np.abs(shap_values[idx]).mean(axis=0)   # mean |SHAP| for this cluster
    cluster_shap[cl] = mean_abs

shap_cluster_df = pd.DataFrame(cluster_shap, index=kept_features)

# Sort features by overall importance across clusters
shap_cluster_df["Overall_MeanAbsSHAP"] = shap_cluster_df.mean(axis=1)
shap_cluster_df = shap_cluster_df.sort_values("Overall_MeanAbsSHAP", ascending=False).drop(columns=["Overall_MeanAbsSHAP"])

# Display full table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("\nMean(|SHAP|) per feature per cluster:")
print(shap_cluster_df.to_string())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

short_names = {
    "Small hydrophobic molecules": "Small\nHydrophobic",
    "Large hydrophilic molecules": "Large\nHydrophilic",
    "Midsized Amphiphilic": "Mid\nAmphiphilic",
    "Hydrophilic drugs": "Hydrophilic"
}

plot_df = shap_cluster_df.rename(columns=short_names)

plt.figure(figsize=(7.5, 4))
ax = sns.heatmap(
    plot_df,
    cmap="viridis",
    linewidths=0.5,
    linecolor="white",
    cbar_kws={"label": "mean(|SHAP|)"}
)

ax.set_title("Cluster-wise Feature Importance", pad=10)
ax.set_xlabel("")
ax.set_ylabel("")

plt.xticks(rotation=0)      # now it fits
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

short_names = {
    "Small hydrophobic molecules": "Small\nHydrophobic",
    "Large hydrophilic molecules": "Large\nHydrophilic",
    "Midsized Amphiphilic": "Mid\nAmphiphilic",
    "Hydrophilic drugs": "Hydrophilic"
}

plot_df = shap_cluster_df.copy()

# Column-wise normalization (0–1 per cluster)
plot_df = (plot_df - plot_df.min()) / (plot_df.max() - plot_df.min() + 1e-12)
plot_df = plot_df.rename(columns=short_names)

plt.figure(figsize=(7.5, 4))
ax = sns.heatmap(
    plot_df,
    cmap="viridis",
    linewidths=0.5,
    linecolor="white",
    cbar_kws={"label": "Normalized mean(|SHAP|) per cluster"}
)

ax.set_title("Normalized Cluster-wise Feature Importance", pad=10)
ax.set_xlabel("")
ax.set_ylabel("")

plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

#VI. BAYESIAN OPTIMIZATION depending on Cluster

In [ ]:
import numpy as np
import pandas as pd

# Drug -> Cluster_Label
drug_to_cluster = drug_df.set_index("Drug")["Cluster_Label"].to_dict()
df_with_descriptors["Cluster_Label"] = df_with_descriptors["Drug"].map(drug_to_cluster)

print(df_with_descriptors["Cluster_Label"].value_counts(dropna=False))

In [ ]:
# Define Criteria of Success from literature of drug release. Large hydrophilic drugs allowed higher burst rate due to its nature.
CRITERIA = {
    "Small hydrophobic molecules": {  # cluster 0
        "burst_max": 0.20,
        "total_min": 0.80,
    },
    "Large hydrophilic molecules": {  # cluster 1
        "burst_max": 0.30,
        "total_min": 0.70,
    },
    "Midsized Amphiphilic": {         # cluster 2
        "burst_max": 0.20,
        "total_min": 0.70,
    },
    "Hydrophilic drugs": {            # cluster 3
        "burst_max": 0.20,
        "total_min": 0.70,
    },
}

# Common sustained-release settings
DURATION_START_DAY = 14.0
DURATION_END_DAY   = 30.0
MIN_GAIN_AFTER_14  = 0.10  # at least +10% release after day 14

In [ ]:
feature_names = list(X_train_top10.columns)
time_cols = [c for c in feature_names if "time" in c.lower()]
assert len(time_cols) == 1, f"Expected exactly 1 time feature. Found: {time_cols}"
TIME_FEATURE = time_cols[0]
print("Time feature =", TIME_FEATURE)

In [ ]:
from pandas.api.types import is_numeric_dtype

# Heuristic: treat these as "drug descriptors" to exclude from optimization
drug_descriptor_like = set([c for c in df_with_descriptors.columns if c.lower().startswith("drug ")])
drug_descriptor_like |= set([c for c in feature_names if c.lower().startswith("drug ")])

# Continuous candidates: numeric model features excluding time and drug descriptors
cont_candidates = []
for c in feature_names:
    if c == TIME_FEATURE:
        continue
    if c in drug_descriptor_like:
        continue
    # try to avoid one-hot method dummies here
    if "formulation method_" in c.lower():
        continue
    # keep numeric
    if c in X_train_top10.columns:
        cont_candidates.append(c)

# Method dummies (optional)
method_dummy_cols = [c for c in feature_names if "formulation method_" in c.lower()]

print("Continuous BO vars (auto):", cont_candidates)
print("Method dummy cols:", method_dummy_cols)

In [ ]:
#Picking representative drugs in each cluster
desc_cols = [c for c in ["Drug LogP", "Drug MW", "Drug TPSA"] if c in drug_df.columns]
assert len(desc_cols) >= 2, f"Need at least 2 drug descriptor columns in drug_df, found: {desc_cols}"

def pick_representative_drug(drug_df, cluster_label):
    sub = drug_df[drug_df["Cluster_Label"] == cluster_label].copy()
    centroid = sub[desc_cols].mean().values
    dists = np.linalg.norm(sub[desc_cols].values - centroid, axis=1)
    return sub.iloc[int(np.argmin(dists))]["Drug"]

clusters = [cl for cl in df_with_descriptors["Cluster_Label"].dropna().unique() if cl in CRITERIA]
rep_drug = {cl: pick_representative_drug(drug_df, cl) for cl in clusters}
rep_drug

In [ ]:
# Align training rows to X_train_top10 index (if you use train_idx elsewhere, you can swap here)
train_df_aligned = df_with_descriptors.loc[X_train_top10.index].copy()

def get_cluster_bounds(X, train_df, cluster_label, vars_cont):
    sub_idx = train_df["Cluster_Label"] == cluster_label
    bounds = {}
    for v in vars_cont:
        lo = float(X.loc[sub_idx, v].min())
        hi = float(X.loc[sub_idx, v].max())
        if not np.isfinite(lo) or not np.isfinite(hi) or lo == hi:
            # fallback: global
            lo = float(X[v].min()); hi = float(X[v].max())
        bounds[v] = (lo, hi)
    return bounds

cluster_bounds = {cl: get_cluster_bounds(X_train_top10, train_df_aligned, cl, cont_candidates) for cl in clusters}
cluster_bounds

In [ ]:
def score_curve(pred_curve, times, cluster_label):
    cfg = CRITERIA[cluster_label]

    # Interpolate burst and day14
    r_day1  = float(np.interp(1.0, times, pred_curve))
    r_day14 = float(np.interp(DURATION_START_DAY, times, pred_curve))
    r_final = float(pred_curve[-1])

    loss = 0.0

    # 1) Burst (soft constraint)
    burst_excess = max(0.0, r_day1 - cfg["burst_max"])
    loss += 25.0 * (burst_excess ** 2)

    # Preder ~0.15 for clusters with 0.20 max (<15–20%)
    if cfg["burst_max"] <= 0.20:
        loss += 2.0 * ((r_day1 - 0.15) ** 2)

    # 2) Total release at end
    total_deficit = max(0.0, cfg["total_min"] - r_final)
    loss += 40.0 * (total_deficit ** 2)

    # 3) Monotonicity
    diffs = np.diff(pred_curve)
    neg = diffs[diffs < 0]
    if len(neg):
        loss += 80.0 * float(np.sum(neg**2))

    # 4) Sustained duration proxy: must keep releasing after day 14
    gain_after_14 = r_final - r_day14
    duration_deficit = max(0.0, MIN_GAIN_AFTER_14 - gain_after_14)
    loss += 15.0 * (duration_deficit ** 2)

    return float(loss)

In [ ]:
def get_base_row_for_drug(df, drug_name):
    sub = df[df["Drug"] == drug_name]
    assert len(sub) > 0, f"No rows for drug: {drug_name}"
    return sub.iloc[0]

def set_method_dummies(row_dict, method_cols, chosen=None):
    # chosen=None means baseline (all zeros)
    for c in method_cols:
        row_dict[c] = 0.0
    if chosen is not None:
        row_dict[chosen] = 1.0

def predict_curve_xgb(model, base_row, params_cont, times, method_cols=None, method_choice=None):
    rows = []
    for t in times:
        r = {c: 0.0 for c in X_train_top10.columns}

        # Fill any features that exist in the base row (drug descriptors etc.)
        for c in X_train_top10.columns:
            if c in base_row.index:
                val = base_row[c]
                if pd.notnull(val):
                    r[c] = float(val)

        # Apply continuous design knobs
        for k, v in params_cont.items():
            if k in r:
                r[k] = float(v)

        # Apply method (if dummies exist)
        if method_cols and len(method_cols):
            set_method_dummies(r, method_cols, method_choice)

        # Set time feature
        r[TIME_FEATURE] = float(t)

        rows.append(r)

    Xq = pd.DataFrame(rows, columns=X_train_top10.columns)
    preds = model.predict(Xq)
    return np.clip(preds, 0.0, 1.0)

#VI. a. Bayesian Optimization per cluster

In [ ]:
!pip -q install scikit-optimize

In [ ]:
from skopt import gp_minimize
from skopt.space import Real, Categorical
from skopt.utils import use_named_args

def run_bo_for_cluster(cluster_label, n_calls=50, n_init=12, seed=1147):
    # Representative drug
    drug_name = rep_drug[cluster_label]
    base_row = get_base_row_for_drug(df_with_descriptors, drug_name)

    # Bounds
    bounds = cluster_bounds[cluster_label]

    # Search space: continuous
    space = [Real(bounds[v][0], bounds[v][1], name=v) for v in cont_candidates]

    # Search space: method dummy choice (optional)
    method_choices = [None] + method_dummy_cols  # None = baseline (all zeros)
    if len(method_dummy_cols):
        space.append(Categorical(method_choices, name="MethodDummy"))

    @use_named_args(space)
    def objective(**kwargs):
        # Split params
        params_cont = {v: kwargs[v] for v in cont_candidates}
        method_choice = kwargs.get("MethodDummy", None)

        pred_curve = predict_curve_xgb(
            model=final_xgb_top10,
            base_row=base_row,
            params_cont=params_cont,
            times=TIMES,
            method_cols=method_dummy_cols,
            method_choice=method_choice
        )

        return score_curve(pred_curve, TIMES, cluster_label)

    res = gp_minimize(
        objective,
        dimensions=space,
        n_calls=n_calls,
        n_initial_points=n_init,
        random_state=seed,
        acq_func="EI"
    )

    # Decode best
    best = {dim.name: val for dim, val in zip(space, res.x)}
    best_cont = {v: best[v] for v in cont_candidates}
    best_method = best.get("MethodDummy", None)

    best_curve = predict_curve_xgb(
        model=final_xgb_top10,
        base_row=base_row,
        params_cont=best_cont,
        times=TIMES,
        method_cols=method_dummy_cols,
        method_choice=best_method
    )

    return {
        "cluster": cluster_label,
        "representative_drug": drug_name,
        "best_loss": float(res.fun),
        "best_continuous_params": best_cont,
        "best_method_dummy": best_method,
        "times": TIMES,
        "best_pred_curve": best_curve,
        "skopt_result": res
    }

bo_results = []
for cl in clusters:
    print(f"\n=== BO for cluster: {cl} | rep drug: {rep_drug[cl]} ===")
    bo_results.append(run_bo_for_cluster(cl, n_calls=50, n_init=12))

bo_results[0].keys()

#VI.b. Best Curve per cluster and reccomendation

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
for r in bo_results:
    plt.plot(r["times"], r["best_pred_curve"], marker="o", label=r["cluster"])
plt.xlabel("Time (days)")
plt.ylabel("Predicted cumulative release")
plt.title("BO-Optimized Sustained Release Curves (per cluster)")
plt.legend()
plt.tight_layout()
plt.show()

for r in bo_results:
    print("\n---", r["cluster"], "---")
    print("Representative drug:", r["representative_drug"])
    print("Best loss:", r["best_loss"])
    print("Best continuous params:")
    for k, v in r["best_continuous_params"].items():
        print(f"  {k}: {v:.4f}")
    if len(method_dummy_cols):
        print("Best method dummy:", r["best_method_dummy"])